<a href="https://colab.research.google.com/github/OCCI-Staar/Curve-Maps-With-Clouds/blob/main/Curve_Maps_SHP_Clouds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note this will **NOT** filter out clouds
## Create a standard environment

*   Simply Click play button to run all scripts

*   If no errors, continue to 1)

In [1]:
#Just need the required versions from GitHub
!wget -O required-info.txt https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-With-Clouds/main/required-info.txt

--2024-06-21 17:54:43--  https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-With-Clouds/main/required-info.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129 [text/plain]
Saving to: ‘required-info.txt’

required-info.txt   100%[===================>]     129  --.-KB/s    in 0s      

2024-06-21 17:54:43 (1.99 MB/s) - ‘required-info.txt’ saved [129/129]



In [2]:
# View the content of required-info.txt
!cat required-info.txt

# Install packages listed in required-info.txt
!pip install -r required-info.txt

pandas==2.0.3 
geopandas==0.14.4 
shapely==2.0.2 
fiona==1.9.5 
pyproj==3.6.1 
geetools==0.6.11 
earthengine-api==0.1.347


In [3]:
#Needed Libraries
import ee
import time
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from datetime import datetime
from geetools import batch
import ipywidgets as widgets

from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt

# 1) Authenticate yourself through Google Earth Engine

In [4]:
#Authenticate the GEE User
ee.Authenticate()
# Verify if GEE server was authenticated
try:
  ee.Initialize()
except ee.EEException as error:
  print(f'ERROR: {error}')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nsHls0ftxEMzFTT0vJMOyhkPTdzbStFCzQGmltJiTOU&tc=curjD2Eoty_o0HQsoZkND2oMNdIC72i591892H5AhWU&cc=VtlBDUowy5Eea51U5eoAWtBMHCPVjpKMdH4d0VA_yd0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ATx3LY4E5x0GAGQ21hHECWmZ9pI-664k1ocG5H5L2p3mPPX-80ag0DUlGR0

Successfully saved authorization token.


# 2) Supply Satellite, Date Range, Field Name

In [5]:
satellite = "COPERNICUS/S2_SR_HARMONIZED" # @param ["LANDSAT/LT04/C01/T1_SR", "LANDSAT/LT05/C01/T1_SR", "LANDSAT/LE07/C01/T1_SR", "LANDSAT/LC08/C01/T1_SR", "COPERNICUS/S2_SR_HARMONIZED"]
first_date = ''  #@param {type: "date"}
last_date  = ''  #@param {type: "date"}
field_name = "" #@param {type:"string"}


if satellite == "COPERNICUS/S2_SR_HARMONIZED":
      sat = 'S2SR'
if satellite == "LANDSAT/LT04/C01/T1_SR":
      sat = 'LT04'
if satellite == "LANDSAT/LT05/C01/T1_SR":
      sat = 'LT05'
if satellite == "LANDSAT/LE07/C01/T1_SR":
      sat = 'LE07'
if satellite == "LANDSAT/LC08/C01/T1_SR":
      sat = 'LC08'

# 3) Choose Field Boundaries file you want processed.


*   File extension must be **.shp**


In [20]:
from google.colab import files
shp_file = files.upload()


#If choosing a file more then once it'll add a numbe extension to the file
base_filename, ext = os.path.splitext(shp_file)  # Split filename into base and extension
normalized_filename = base_filename.rsplit('(', 1)[0].strip() + ext
print(normalized_filename)

Saving Gatez_L1_Boundary_poly.shp to Gatez_L1_Boundary_poly (4).shp


TypeError: expected str, bytes or os.PathLike object, not dict


*   Now choose **.shx** extension

In [7]:
shx_file = files.upload()

Saving Gatez_L1_Boundary_poly.shx to Gatez_L1_Boundary_poly (1).shx


*   Now choose **.dbf** extension

In [8]:
dbf_file = files.upload()

Saving Gatez_L1_Boundary_poly.dbf to Gatez_L1_Boundary_poly (1).dbf


*   Now choose **.prj** extension

In [9]:
prj_file = files.upload()

Saving Gatez_L1_Boundary_poly.prj to Gatez_L1_Boundary_poly (1).prj


All files chosen?

In [17]:
shp_path = next(iter(shp_file))
shx_path = next(iter(shx_file))
dbf_path = next(iter(dbf_file))
prj_path = next(iter(prj_file))

base_filename = shp_path.split('.shp')[0]
bound_files = []
bound_files = [shp_path, shx_path, dbf_path, prj_path]

# 4) Time to process and export images

In [14]:
#With Clouds
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import satellite info
    s2_sr_col = (ee.ImageCollection(satellite)
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    return s2_sr_col

In [15]:
#Pulls geometry from the polygon
def get_bbox_coords(pol):
    shp_bb = base_filename  + '_bb.geojson'
    #Convert polygon CRS to WGS84 DD
    pol = pol.to_crs({'init': 'epsg:4326'})
    #Create a buffer for the boundary and save as geoJSON
    pol.envelope.to_file(filename=shp_bb, driver="GeoJSON")
    #Open GeoJSON
    geom = json.loads(pol.envelope.to_json())['features'][0]['geometry']

    return(geom)

In [18]:
for bound_file in bound_files:
  pol = gpd.read_file(bound_file)

  pol = pol.dissolve()

  pol = pol.buffer(0.005)

  geom = get_bbox_coords(pol)

  #Get coordinates from GeoJSON
  coords = geom['coordinates']
  regions = ee.Geometry.Polygon(coords)

  start_date = ee.Date(first_date)
  end_date = ee.Date(last_date)

  if sat == 'S2SR':
    scale = 10
    extra = dict(sat=sat)
    s2_sr_cld_col = get_s2_sr_cld_col(regions, start_date, end_date)
    imagery = s2_sr_cld_col

field_name_sat = f'{field_name}'+"{id}"

print(imagery.size().getInfo())
batch.Export.imagecollection.toDrive(imagery, folder=f'YieldCurve_{sat}_{field_name}',
                                     region=regions, scale=scale, verbose = True,
                                     namePattern = field_name_sat)
time.sleep(300)
print("Export tasks were a success!")

ERROR:fiona._env:Unable to open Gatez_L1_Boundary_poly (2).shx or Gatez_L1_Boundary_poly (2).SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.


DriverError: Unable to open Gatez_L1_Boundary_poly (2).shx or Gatez_L1_Boundary_poly (2).SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.